In [18]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, types, functions as F

In [2]:
spark = SparkSession\
  .builder\
  .master("local[*]")\
  .appName('test')\
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 23:09:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('./data/pq/green/*/*')

In [13]:
df_green = df_green\
  .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')\
  .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [4]:
df_yellow = spark.read.parquet('./data/pq/yellow/*/*')

In [14]:
df_yellow = df_yellow\
  .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')\
  .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [16]:
common_columns = [col for col in df_green.columns if col in df_yellow.columns]

In [19]:
df_green_sel = df_green.select(common_columns).withColumn('service_type', F.lit('green'))
df_yellow_sel = df_yellow.select(common_columns).withColumn('service_type', F.lit('yellow'))

In [20]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [21]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2752287|
|      yellow|39649199|
+------------+--------+



In [23]:
df_trips_data.createOrReplaceTempView('trip_data')

In [ ]:
df_result = spark.sql("""
  SELECT 
    PULocationID AS revenue_zone,
    date_trunc("month", "pickup_datetime") AS revenue_month,
    service_type, 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance
  FROM trip_data
  GROUP BY revenue_zone, revenue_month, service_type
""")

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `pickup_zone` cannot be resolved. Did you mean one of the following? [`pickup_datetime`, `mta_tax`, `tip_amount`, `VendorID`, `extra`].; line 3 pos 4;
'Aggregate ['revenue_zone, 'revenue_month, service_type#279], ['pickup_zone AS revenue_zone#421, date_trunc(month, cast(pickup_datetime as timestamp), Some(Europe/Luxembourg)) AS revenue_month#422, service_type#279, sum(fare_amount#9) AS revenue_monthly_fare#423, sum(extra#10) AS revenue_monthly_extra#424, sum(mta_tax#11) AS revenue_monthly_mta_tax#425, sum(tip_amount#12) AS revenue_monthly_tip_amount#426, sum(tolls_amount#13) AS revenue_monthly_tolls_amount#427, sum(improvement_surcharge#15) AS revenue_monthly_improvement_surcharge#428, sum(total_amount#16) AS revenue_monthly_total_amount#429, avg(passenger_count#7) AS avg_monthly_passenger_count#430, avg(trip_distance#8) AS avg_monthly_trip_distance#431]
+- SubqueryAlias trip_data
   +- View

In [ ]:
spark.stop()